explore ways to use "weights" array in RMS and Correlation functions of CDAT

CDAT functions require that
1. weights is an MV2 type variable
2. weights array has "bounds" attribute on each dimensions
3. the dimension ids of the weights array match the dimensions of the fields that are being evaluated  

This can be achieved by 
1. turning on autobounds for a bit
2. making a copy of an array to be evaluated as the weight array, 
3. turning off autobounds
4. then replacing the id, and the data (with the chosen weights).




In [1]:
%matplotlib inline
import numpy as np
import cdms2
import cdutil
from cdms2 import MV2
from genutil import statistics

In [43]:
jname1 = "/Users/d3x345/Desktop/NetCDF_files/vd05_ANN_climo.nc"
print "jname1", jname1

print "info before calling autobounds"
g1 = cdms2.open(jname1)
lat_bnds = g1("lat_bnds")
lev_bnds = 1000.*(g1("hyai")+g1("hybi"))
#print "lat_bnds",lat_bnds[0:4]
#print "lev_bnds bot",lev_bnds.sum(), lev_bnds[-4:]
#print "lev_bnds", lev_bnds[0:4]
mypwts = lev_bnds[1:]-lev_bnds[0:-1]
#print "mypwts", mypwts.shape, mypwts.sum(), mypwts
#print "mypwts", mypwts[0:4]

T1 =  g1("T",squeeze=1)
lat = T1.getAxis(1)
T1xav = cdutil.averager(T1,axis="x")

T2 =  g1("T",squeeze=1)
T2xav = cdutil.averager(T2,axis="x")

# now create a bounds using autobounds
orig_bounds = cdms2.getAutoBounds()
#print "autobounds", orig_bounds
cdms2.setAutoBounds(1)
#print "autobounds set"

lev = T2xav.getLevel()
#print "autobounds2", cdms2.getAutoBounds()


Tw2 = cdutil.area_weights(T2xav)
#print "Tw2.shape", Tw2.shape, Tw2[0:4,0], Tw2[0,0:4]
#print "Tw2.sum", Tw2.sum()

lat1 = T1xav.getAxis(1)
lbnd =lat1.getBounds()
#print "lbnd", lbnd.shape,lbnd[0:4,1], lbnd[0:4,0]
myywts = (np.sin(lbnd[:,1]*np.pi/180.)-np.sin(lbnd[:,0]*np.pi/180.))*0.5
#print "myywts", myywts[0:4]

#dlat = lbnd[:,1]-lbnd[:,0]
#print "dlat", dlat[0:4]
levs1 = T1xav.getAxis(0)
# looks like CDMS puts bounds halfway between dimension values,
# and then does something reasonable with the endpoints
bnd=levs1.getBounds()
#print "bnd.shape", bnd.shape
#print "cdms guess at bnd", bnd
#db = bnd[:,1]-bnd[:,0]
#print "db", db[0:4]

#print "db.shape", db.shape, db.sum()
#1./0.
levs1.setBounds(bnd)
# overwrite the default bounds with my version
levs1.setBounds(lev_bnds)
#print "lev_bnds 2", lev_bnds[0:4]
bnd=levs1.getBounds()
#print "bnd 2", bnd
db = bnd[:,1]-bnd[:,0]
#print "db 2", db[0:4]
#print "levs1 db",db[0:4]
cdms2.setAutoBounds(orig_bounds)
#print "autobounds unset"
#print "autobounds3", cdms2.getAutoBounds()

pert = 10.*np.random.random_sample((72, 192))
T1xav = T1xav + pert
levwts = levs1.getData()
levwts = db
#levwts[:] = 1.
#levwts = levwts/levwts.sum()
print "levwts.sum", levwts.dtype, levwts.sum()
print "levwts", levwts[0:4]

#print "help T1xav", help(T1xav)
tweights = T1xav.copy()
#print "tw info", tweights.info()

grid = T1.getGrid()
latwts, lonwts = grid.getWeights()
print "latwts", latwts[0:4]


#exweights = MV2.outerproduct(latwts, lonwts)
#print "exweights", exweights
#print "exweights.info", exweights.info()
#latwts[:] = 1.
#print "latwts.sum",latwts.dtype, latwts.sum()



jname1 /Users/d3x345/Desktop/NetCDF_files/vd05_ANN_climo.nc
info before calling autobounds
levwts.sum float64 999.8999999968024
levwts [0.04765082 0.07035683 0.10388243 0.15338325]
latwts [1.69087141e-05 1.35262851e-04 2.70489109e-04 4.05642190e-04]


In [54]:
weights = MV2.outerproduct(levwts, latwts)
print "levwts[0],latwts[0],product", levwts[0], latwts[0], levwts[0]*latwts[0]
tweights.data[:] = weights.data[:]
tweights.id = 'tweights'
print "tweights.shape", tweights.shape, tweights[0:4,0], tweights[0,1:4]
#tweights = tweights/tweights.sum()
print "tweights.sum", tweights.sum()

# correlation works with no weights
result = statistics.correlation(T1xav, T2xav, axis='zy')
print "unweighted correlation", result
result = statistics.correlation(T1xav, T2xav, weights=tweights, axis='zy')
print "weighted correlation", result
result = statistics.rms(T1xav, T2xav, axis='zy')
print "unweighted rms", result
result = statistics.rms(T1xav, T2xav, weights=tweights, axis='zy')
print "weighted rms", result

#result = statistics.correlation(T1xav, T2xav, weights=(levwts,latwts), axis='zy')
#print "weighted correlation with 1D arrays, result", result

levwts[0],latwts[0],product 0.04765081971614585 1.690871407966643e-05 8.057140862420421e-07
tweights.shape (72, 192) [8.057140862420421e-07 1.18964344062853e-06 1.7565182736613226e-06
 2.593513602761546e-06] [6.445385727494766e-06 1.2889027753520937e-05 1.932918284834894e-05]
tweights.sum 999.8999999968024
unweighted correlation 0.9951268872275159
weighted correlation 0.9946999409570033
unweighted rms 5.799573175788458
weighted rms 5.812968754200054
